# Green PPA Pricing - Expected Value

In [7]:
import sys
import datetime as dt
sys.path.append('../../../..')
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import logging
#logging.basicConfig(level=logging.DEBUG, 
#                    format="%(asctime)s  - %(levelname)s - %(filename)s:%(lineno)s - %(message)s ")
logger = logging.getLogger('rivapy.pricing')
logger.setLevel(logging.DEBUG)

from rivapy.instruments import SimpleSchedule,  GreenPPASpecification
#from rivapy.models.residual_demand_fwd_model import WindPowerForecastModel, WindPowerForecastModelParameter, MultiRegionWindForecastModel, LinearDemandForwardModel

import rivapy.sample_data.residual_demand_models as rdm
from rivapy.tools.datetime_grid import DateTimeGrid

from rivapy.marketdata import PowerPriceForwardCurve
import rivapy.sample_data.residual_demand_models as rdm_sample

from rivapy.instruments.factory import create
from rivapy.pricing import green_ppa_mean_pricer

from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

%load_ext autoreload
%autoreload 2

%matplotlib inline

C:\Users\doeltz\development\RiVaPy\docs\source\notebooks\pricing\../../../..\rivapy\__init__.py:11: UserWarning: The pyvacon module is not available. You may not use all functionality without this module. Consider installing pyvacon.
  warnings.warn('The pyvacon module is not available. You may not use all functionality without this module. Consider installing pyvacon.')


## Green PPA Specification

In [2]:
simple_schedule = SimpleSchedule(dt.datetime(2022,12,1), dt.datetime(2023,12,1,4,0,0), freq='1H')
green_ppa = GreenPPASpecification(simple_schedule, 
                                  fixed_price = 10.0, 
                                  max_capacity=10, 
                                  technology = 'wind', 
                                  udl = 'power',
                                  location='')

In [4]:
results = rd_model.simulate(dg, n_sims=1000, start_value_wind=0.5, 
                            start_value_load=0.5, start_value_solar=0.5, seed=42)

NameError: name 'rd_model' is not defined

In [3]:
green_ppa_mean_pricer.price(dt.datetime(2022,12,1), green_ppa, rd_model)

NameError: name 'rd_model' is not defined

In [41]:
','.join(['q','f'])

'q,f'

In [4]:
from enum import Enum
class PricingRequestTypes(Enum):
    PRICE = 'PRICE'

    
class PricingRequest:
        
    def __init__(self):
        #print(type(self)._validate())
        for k in type(self)._get_choices():
            setattr(self,k,False)
        depp = ', '.join(type(self).get_choices())
        self.__doc__='Container to store the information which results the respective pricer shall compute. \n Following attributes are provided: '+depp 
        
    @classmethod
    def _validate(cls):
        return cls.selections
    
    @classmethod
    def set_request(cls, x):
        cls.selections.append(x)
        
    @classmethod
    def _get_choices(cls):
        return cls.selections
    
class B(PricingRequest):
    selections = ['A','B']
    
class C(PricingRequest):
    selections = ['C','D']
    
b=B()
c = C()
c.set_request('A')
d = B()
c.set_request('B')

In [9]:
from rivapy.pricing.pricing_request import GreenPPAPricingRequest

In [59]:
np.sum(np.multiply(results[green_ppa.technology],results['price']),axis=0)

array([1458838.01658713,  912878.65079862, 1300659.61324367,
       1187527.04573404, 1461813.30606483, 1450109.23215055,
       1210193.93137315, 1392645.8144648 , 1510030.58710112,
       1438963.69229254, 1335710.63758983, 1597345.67265557,
       1254635.06972083, 1282010.90691649, 1417328.6021639 ,
       1413159.02346185, -224329.44074632, 1324335.19556399,
       1297913.89454097, 1107460.90084507, 1376237.2649392 ,
       1496726.56860321, 1310536.71366256, 1002753.27276074,
       1386327.28126734, 1337377.25203679, 1293562.15791379,
       1280815.61864716, 1317016.5468621 , 1031152.91403738,
       1521380.38528176, 1387796.17325859, 1056142.7809435 ,
       1203768.64299416, 1222044.46263714, 1286765.32248347,
       1381959.06316813, 1412614.4853771 , 1418830.32385771,
       1614218.76647742, 1236638.5678654 , 1618001.43452123,
       1218830.9993063 , 1306888.94429735, 1505138.01546509,
        977000.06682582, 1444303.51671722, 1154569.78741383,
       1247690.54734613,

In [55]:
results['price']

(8764, 100)

In [11]:
df = dg.df
df[df.dates.isin([dt.datetime(2022,12,1)])].index.values

array([0], dtype=int64)

In [ ]:
values = np.ones((len(simple_schedule.get_schedule()),))
hpfc = PowerPriceForwardCurve( dt.datetime(2022,12,1), dt.datetime(2022,12,1), 
                              dt.datetime(2023,12,1,4,0,0), freq='1H', values = values)